In [2]:
import csv
import pandas as pd
import datetime

In [3]:
readers = pd.DataFrame([], columns = ['date', 'action', 'country', 'user_id', 'source', 'topic']) 
returning_readers = pd.DataFrame([], columns = ['date', 'action', 'country', 'user_id', 'topic']) 

purchases = pd.DataFrame([], columns = ['date', 'action', 'user_id', 'spent']) 
subscribes = pd.DataFrame([], columns = ['date', 'action', 'user_id']) 

#### Read in file

In [4]:
with open("dilans_data.csv") as input_file:
    readerCSV = csv.reader(input_file)

    readers_list = []
    returning_list = []
    purchases_list = []
    subscribes_list = []
    for row in enumerate(readerCSV):
        action =row[1][0].split(';')[1]
        r = row[1][0].split(';')
        if action == 'read':
            if len(r) == 6:
                readers_list.append(r)
            else:
                returning_list.append(r)
        if action == 'subscribe':
            subscribes_list.append(r)
        if action == 'buy':
            purchases_list.append(r)
        
    readers=pd.DataFrame(readers_list,columns=['date', 'action', 'country', 'user_id', 'source', 'topic'])
    returning_readers=pd.DataFrame(returning_list,columns=['date', 'action', 'country', 'user_id', 'topic'])
    purchases=pd.DataFrame(purchases_list,columns=['date', 'action', 'user_id', 'spent'])
    subscribes=pd.DataFrame(subscribes_list,columns=['date', 'action', 'user_id'])

### Highest revenue

In [5]:
travels = purchases.set_index('user_id').join(readers.set_index('user_id'), how='left', lsuffix='_purchases', rsuffix= '_readers')

In [6]:
travels[['spent']] = travels[['spent']].apply(pd.to_numeric)
print(type(travels.iloc[0].spent))

travels.groupby(by='country')['spent'].sum()


<class 'numpy.int64'>


country
country_1      920
country_2    39528
country_3     3616
country_4    33464
country_5    57920
country_6    16432
country_7    39760
country_8     2840
Name: spent, dtype: int64

##### Highest revenue per month

In [7]:
readers['date'] = pd.to_datetime(readers['date'])
returning_readers['date'] = pd.to_datetime(readers['date'])
purchases['date'] = pd.to_datetime(purchases['date'])
subscribes['date'] = pd.to_datetime(subscribes['date'])
travels['date_purchases'] = pd.to_datetime(travels['date_purchases'])

In [8]:
travels['month'] = travels['date_purchases'].apply(lambda x : x.month)

In [10]:
travels.sort_values(ascending = False, by = 'country').groupby(['country', 'month']).agg('sum')

spent
country   month       
country_1 1         96
          2        392
          3        432
country_2 1       2656
          2      12776
          3      24096
country_3 1        472
          2       1528
          3       1616
country_4 1       7136
          2      10456
          3      15872
country_5 1       8880
          2      18488
          3      30552
country_6 1        984
          2       4288
          3      11160
country_7 1       2776
          2      13968
          3      23016
country_8 1        136
          2        376
          3       2328

#### Summary by sold videos and books

In [68]:
books = travels.loc[travels['spent'] == 8].groupby('country').agg('count')[['spent']]
videos = travels.loc[travels['spent'] == 80].groupby('country').agg('count')[['spent']]

pandas.core.frame.DataFrame

In [76]:
books

,spent
country,
country_1,25
country_2,1441
country_3,112
country_4,1093
country_5,1970
country_6,534
country_7,1380
country_8,85


In [77]:
videos

,spent
country,
country_1,9
country_2,350
country_3,34
country_4,309
country_5,527
country_6,152
country_7,359
country_8,27
